### 测试对抗样本生成效果

In [1]:
import torch
import random
import numpy as np
import pandas as pd
import numpy as np
import foolbox as fb
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F

/root/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
log_loss_file=('./loss_log/mnist_LogisticRegressor_ntp_60000_seed_0_epochs_10_lr_0.01_wd_0.01_bs_256_optim_sgd_notransform_loss.csv')
loss_df = pd.read_csv(log_loss_file)
loss_df1 = pd.read_csv(log_loss_file)
loss_df.sort_values('loss', inplace=True, ascending=False)
loss_df1.sort_values('max', inplace=True)

In [9]:
loss_df[:30]

,max,max_diff,loss,gradnorm,y_true,y_pred
24798,0.999414,0.999142,15.455508,50.742398,4,0
8729,0.964718,0.929619,14.403582,45.573702,3,7
32776,0.470431,0.017194,12.877142,34.846987,4,2
15942,0.622972,0.343417,12.786316,38.767208,4,2
26882,0.999136,0.998399,12.730476,46.971258,7,0
50514,0.987892,0.977008,12.526046,45.571205,8,3
13969,0.844480,0.761259,12.328907,37.271516,3,6
52358,0.794660,0.730476,12.324869,35.364943,2,4
18598,0.889307,0.827378,11.785258,50.020748,9,2
12679,0.977994,0.961801,11.594181,47.385070,2,8


: 

In [3]:
ordering = loss_df.index.tolist()

In [4]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, RandomCrop, RandomHorizontalFlip
train_transformation = Compose([
            ToTensor(),
            Normalize((0.1307,), (0.3081,)),
        ])
train_dataset = MNIST(root='./data', train=True, download=True, transform=train_transformation)

/root/anaconda3/envs/py39/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/anaconda3/envs/py39/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
from models import LogisticRegressor
import os
# import foolbox as fb
device = torch.device("cuda")
model = LogisticRegressor().cuda()
model.load_state_dict(torch.load("./trained_models/mnist_LogisticRegressor_ntp_60000_seed_0_epochs_10_lr_0.01_wd_0.01_bs_256_optim_sgd_notransform.pt",map_location=torch.device('cpu')))
fmodel = fb.PyTorchModel(model, bounds=(-0.4243, 2.8216))

attack = fb.attacks.BoundaryAttack(steps=1000)
# epsilons = [0.0, 0.001, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]
# _, advs, success = attack(fmodel, images, labels, epsilons=epsilons)

/root/anaconda3/envs/py39/lib/python3.9/site-packages/foolbox/models/pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(


In [6]:
loss_df[:20]

,max,max_diff,loss,y_true,y_pred
43534,0.190383,0.005919,1.690300,8,5
53248,0.206846,0.013306,1.649633,8,2
31941,0.209251,0.020093,2.332655,4,5
41572,0.211033,0.008850,3.055791,6,8
34542,0.213539,0.025656,2.352729,1,9
45576,0.219966,0.002706,1.701481,8,5
16964,0.220727,0.015450,1.510830,8,8
43062,0.221132,0.010964,1.559852,8,9
54883,0.221831,0.003623,1.607535,5,1
45348,0.226111,0.004921,1.508731,2,3


In [7]:
ordering_dataset = Subset(train_dataset, ordering[:20])
attack_loader = torch.utils.data.DataLoader(ordering_dataset, batch_size=1, pin_memory=True,
                                              shuffle=False, num_workers=4)
for image, label in attack_loader:
    print(label)
    image, label = image.to(device), label.to(device)
    logits = model(image)
    y_pred = F.softmax(logits, dim=1)
    top_probs, y_top = torch.topk(y_pred, 1)
    y_pred = y_top[0]
    print(y_pred)
    epsilons = [0.0, 0.001, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]
    _, advs, success = attack(fmodel, image, y_pred, epsilons=epsilons)
    print(success)

tensor([8])
tensor([5], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]], device='cuda:0')
tensor([8])
tensor([2], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True]], device='cuda:0')
tensor([4])
tensor([5], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True]], device='cuda:0')
tensor([6])
tensor([8], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False]], device='cuda:0')
tensor([1])
tensor([9], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True]], device='cuda:0')
tensor([8])
tensor([5], d